### NBA Stats Project

In [1]:
#import necessary libraries
import pandas as pd
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pygsheets
import datetime

In [2]:
date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

### Get Player Data
Per Game & Totals. Then concatenate them together

In [3]:
#create a dataframe from the list of dataframes pulls from pd.read_html
df_player = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2020_per_game.html')[0]

In [4]:
#select numeric columns to be used in converting to float
num_cols = df_player.columns.drop(['Player','Pos','Age','Tm','G','GS'])
df_player[num_cols] = df_player[num_cols].apply(pd.to_numeric, errors='coerce')
df_player = df_player[df_player.Rk > 0]
df_player = df_player.groupby('Player').first().reset_index()

In [5]:
#df_player['Player'].value_counts().sort_values(ascending=False).head(10)

In [6]:
#pull in the totals and not just the per game
df_player_total = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2020_totals.html')[0]

In [7]:
#select numeric columns to be used in converting to float
num_cols = df_player_total.columns.drop(['Player','Pos','Age','Tm','G','GS'])
df_player_total[num_cols] = df_player_total[num_cols].apply(pd.to_numeric, errors='coerce')
df_player_total = df_player_total[['Player','Rk','MP','FG','FGA','3P','3PA','FT',
                                   'FTA','TRB','AST','STL','BLK','TOV','PF',
                                   'PTS']]
df_player_total = df_player_total[df_player_total.Rk > 0]
df_player_total = df_player_total.groupby('Player').first().reset_index()

In [8]:
df_combo = df_player.merge(df_player_total, on='Player', suffixes=('_Per','_Tot'))

### Get Advanced Stats

In [9]:
#create a dataframe from the list of dataframes pulls from pd.read_html
df_adv = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2020_advanced.html')[0]

In [10]:
#wrangle the dataframe
df_adv = df_adv.drop(['Unnamed: 19','Unnamed: 24'], axis=1)
num_cols = df_adv.columns.drop(['Player','Pos','Age','Tm','G'])
df_adv[num_cols] = df_adv[num_cols].apply(pd.to_numeric, errors='coerce')
df_adv = df_adv[['Rk','Player','PER','TS%','3PAr',
                 'FTr','ORB%','DRB%','TRB%','AST%',
                 'STL%','BLK%','TOV%','USG%']]
df_adv = df_adv[df_adv.Rk > 0]
df_adv = df_adv.groupby('Player').first().reset_index()

In [11]:
df_combo = df_combo.merge(df_adv, on='Player', suffixes=('_M1','Adv'))

### Get Per 100 Possessions Stats

In [12]:
#create a dataframe from the list of dataframes pulls from pd.read_html
df_100 = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2020_per_poss.html')[0]

In [13]:
#wrangle the dataframe
df_100 = df_100[['Rk','Player','ORtg','DRtg','PTS']]
num_cols = df_100.columns.drop(['Player'])
df_100[num_cols] = df_100[num_cols].apply(pd.to_numeric, errors='coerce')
df_100 = df_100[df_100.Rk > 0]
df_100 = df_100.groupby('Player').first().reset_index()

In [14]:
df_combo = df_combo.merge(df_100, on='Player', suffixes=('_M1','100'))
df_combo = df_combo.drop(['Rk100','Rk_M1','Rk_Tot','Rk_Per'], axis=1)
df_combo = df_combo.rename(columns={'PTS':'PTS_100'})
df_combo['UpdateTime'] = date

In [22]:
df_combo.tail()

,Player,Pos,Age,Tm,G,GS,MP_Per,FG_Per,FGA_Per,FG%,...,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,PTS_100,UpdateTime
471,Yuta Watanabe,SG,25,MEM,4,0,4.5,0.8,1.5,0.500,...,14.8,0.0,0.0,4.8,14.3,16.2,100.0,112.0,18.1,2020-01-02 16:21:55
472,Zach Collins,PF,22,POR,3,3,28.7,3.0,6.3,0.474,...,7.1,9.3,0.0,1.9,21.7,13.6,112.0,117.0,14.8,2020-01-02 16:21:55
473,Zach LaVine,SG,24,CHI,34,34,33.3,7.9,18.5,0.427,...,7.3,20.9,1.9,1.2,13.9,30.7,104.0,107.0,33.4,2020-01-02 16:21:55
474,Zach Norvell,SG,22,LAL,2,0,2.5,0.0,0.5,0.000,...,11.1,0.0,0.0,0.0,0.0,8.5,0.0,109.0,0.0,2020-01-02 16:21:55
475,Zylan Cheatham,SF,24,NOP,2,0,10.0,0.5,2.0,0.250,...,7.9,6.2,0.0,0.0,20.0,10.3,53.0,117.0,4.7,2020-01-02 16:21:55


### Connect to Google Sheets

In [16]:
pycred = pygsheets.authorize(service_file='credentials2.json')
#opening the gsheet and sheet you want to work with
ss = pycred.open('NBA Stats 2020')[0]
#overwrite what is in the sheet with your df
ss.set_dataframe(df_combo,(1,1))

### Get Team Data

In [17]:
#create a dataframe from the list of dataframes pulls from pd.read_html
df_team = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2020_ratings.html')[0]
#remove the multi-index column header
df_team.columns = df_team.columns.droplevel()

In [18]:
#opening the gsheet and sheet you want to work with
ss2 = pycred.open('NBA Stats 2020')[1]
#overwrite what is in the sheet with your df
ss2.set_dataframe(df_team,(1,1))

### Get per game data from the highest scorers in NBA history

In [19]:
#url_list = []

#for i in range(2008,2020):
#    urls = 'https://www.basketball-reference.com/players/j/duranke01/gamelog/%d' %i
#    data = pd.read_html(urls)[7]
#    #data = data[['Rk','G','Date','']]
#    data['Year'] = i
#    url_list.append(data)

In [20]:
#lebron_data = pd.concat(url_list, ignore_index=True)